In [1]:
import numpy as np
import pandas as pd
import networkx as nx

In [2]:
name = 'bioplex'

In [3]:
omics = pd.read_csv('../PCC/biological_features.csv',sep='\t')

In [4]:
omics_gene = omics['Hugosymbol']

In [5]:
omics

,Hugosymbol,MF: KIRC,MF: BRCA,MF: READ,MF: PRAD,MF: STAD,MF: HNSC,MF: LUAD,MF: THCA,MF: BLCA,...,GE: LUAD,GE: THCA,GE: BLCA,GE: ESCA,GE: LIHC,GE: UCEC,GE: COAD,GE: LUSC,GE: CESC,GE: KIRP
0,STIM1,0.000000,0.005282,0.000000,0.000000,0.012993,0.000000,0.060833,0.003601,0.000000,...,0.029814,0.003233,0.103207,0.064889,0.045089,0.112840,0.090288,0.030371,0.090995,0.017345
1,TRPC1,0.000000,0.005302,0.000000,0.022758,0.000000,0.021187,0.060202,0.000000,0.000000,...,0.060530,0.072560,0.194910,0.046192,0.024565,0.328099,0.076779,0.066241,0.330876,0.042482
2,NOS1,0.063345,0.015765,0.055677,0.022690,0.048721,0.021051,0.038974,0.000000,0.088398,...,0.542696,0.485485,0.000000,0.202997,0.000000,0.264075,0.197229,0.225654,0.338948,0.512714
3,ATP2B4,0.038215,0.014866,0.053870,0.045420,0.025206,0.016322,0.040063,0.007193,0.057544,...,0.094598,0.015496,0.148003,0.011125,0.054450,0.155201,0.071188,0.090702,0.267903,0.028759
4,ABCC9,0.012769,0.031122,0.055910,0.045414,0.187905,0.048539,0.058909,0.010787,0.090623,...,0.185518,0.113508,0.254895,0.071600,0.160738,0.475115,0.056336,0.225954,0.538689,0.096150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13622,GPR153,0.000000,0.010610,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.070449,0.067775,0.062533,0.028065,0.047556,0.164696,0.004251,0.050172,0.015300,0.043336
13623,HDC,0.000000,0.015932,0.000000,0.022764,0.026045,0.005501,0.040862,0.003603,0.030570,...,0.171581,0.061087,0.308353,0.426081,0.109768,0.255002,0.173477,0.188892,0.267474,0.079995
13624,CSMD1,0.062828,0.061911,0.108880,0.157486,0.346369,0.085493,0.332902,0.010780,0.256366,...,0.277948,0.275907,0.384557,0.214808,0.038045,0.190289,0.000000,0.041586,0.341778,0.262938
13625,BHLHE22,0.000000,0.005291,0.000000,0.000000,0.025961,0.000000,0.000000,0.000000,0.000000,...,0.019065,0.252804,0.172442,0.004494,0.152121,0.038414,0.032083,0.091971,0.054766,0.150769


In [6]:
ppi_net = pd.read_csv('data/'+name+'_ppi.csv')
ppi_net = ppi_net[ppi_net['gene1'].isin(omics_gene)]
ppi_net = ppi_net[ppi_net['gene2'].isin(omics_gene)]
ppi_gene = set(ppi_net['gene1']).union(set(ppi_net['gene2']))
omics = omics[omics['Hugosymbol'].isin(ppi_gene)]
omics_gene = omics['Hugosymbol']

In [7]:
import torch
from scipy.sparse import coo_matrix

In [8]:
unique_genes = omics_gene
gene_id_mapping = {gene: i for i, gene in enumerate(omics_gene)}

In [9]:
def sys_normalized_adjacency(adj):
    adj = sp.coo_matrix(adj)
    adj = adj + sp.eye(adj.shape[0])
    row_sum = np.array(adj.sum(1))
    row_sum=(row_sum==0)*1+row_sum
    d_inv_sqrt = np.power(row_sum, -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
    return d_mat_inv_sqrt.dot(adj).dot(d_mat_inv_sqrt).tocoo()

def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

In [10]:
ppi_net['gene1_id'] = ppi_net['gene1'].map(gene_id_mapping)
ppi_net['gene2_id'] = ppi_net['gene2'].map(gene_id_mapping)
gene1_indices = ppi_net['gene1_id'].to_numpy()
gene2_indices = ppi_net['gene2_id'].to_numpy()

ppi_edge_index = torch.tensor([gene1_indices,gene2_indices],dtype=torch.long)

/tmp/ipykernel_13110/494955344.py:6: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1670525552843/work/torch/csrc/utils/tensor_new.cpp:230.)
  ppi_edge_index = torch.tensor([gene1_indices,gene2_indices],dtype=torch.long)


## Omics基因转数字

In [11]:
feature = omics.copy()
feature['Hugosymbol'] = feature['Hugosymbol'].map(gene_id_mapping)
feature

,Hugosymbol,MF: KIRC,MF: BRCA,MF: READ,MF: PRAD,MF: STAD,MF: HNSC,MF: LUAD,MF: THCA,MF: BLCA,...,GE: LUAD,GE: THCA,GE: BLCA,GE: ESCA,GE: LIHC,GE: UCEC,GE: COAD,GE: LUSC,GE: CESC,GE: KIRP
0,0,0.000000,0.005282,0.000000,0.000000,0.012993,0.000000,0.060833,0.003601,0.000000,...,0.029814,0.003233,0.103207,0.064889,0.045089,0.112840,0.090288,0.030371,0.090995,0.017345
2,1,0.063345,0.015765,0.055677,0.022690,0.048721,0.021051,0.038974,0.000000,0.088398,...,0.542696,0.485485,0.000000,0.202997,0.000000,0.264075,0.197229,0.225654,0.338948,0.512714
3,2,0.038215,0.014866,0.053870,0.045420,0.025206,0.016322,0.040063,0.007193,0.057544,...,0.094598,0.015496,0.148003,0.011125,0.054450,0.155201,0.071188,0.090702,0.267903,0.028759
4,3,0.012769,0.031122,0.055910,0.045414,0.187905,0.048539,0.058909,0.010787,0.090623,...,0.185518,0.113508,0.254895,0.071600,0.160738,0.475115,0.056336,0.225954,0.538689,0.096150
5,4,0.000000,0.000000,0.000000,0.022766,0.000000,0.005498,0.000000,0.000000,0.000000,...,0.008233,0.051578,0.155792,0.255082,0.103416,0.075345,0.068497,0.075607,0.038807,0.018624
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13609,10055,0.025655,0.021223,0.000000,0.000000,0.090877,0.027446,0.061204,0.003601,0.061034,...,0.381063,0.027100,0.000000,0.284556,0.015555,0.114083,0.250672,0.437965,0.249747,0.584376
13610,10056,0.000000,0.005313,0.000000,0.022771,0.012818,0.032747,0.020476,0.007205,0.000000,...,0.116118,0.066986,0.065817,0.102184,0.051655,0.095183,0.049556,0.192304,0.237743,0.028278
13613,10057,0.000000,0.005309,0.000000,0.022772,0.000000,0.000000,0.020454,0.000000,0.030760,...,0.230976,0.114448,0.227348,0.133966,0.162193,0.152941,0.100121,0.239105,0.117256,0.158761
13620,10058,0.000000,0.005311,0.000000,0.022761,0.025157,0.005483,0.060276,0.006967,0.000000,...,0.029404,0.054590,0.300185,0.264564,0.108188,0.144933,0.121394,0.186160,0.295876,0.073879


In [12]:
with open('output/'+name+'_gene_id_mapping.txt', 'w') as f:
    for key, value in gene_id_mapping.items():
        f.write(f'{key}: {value}\n')

In [13]:
feature.reset_index(drop=True,inplace=True)
feature.set_index("Hugosymbol",inplace=True)

In [14]:
x = torch.tensor(feature.values, dtype=torch.float)

In [15]:
x

tensor([[0.0000, 0.0053, 0.0000,  ..., 0.0304, 0.0910, 0.0173],
        [0.0633, 0.0158, 0.0557,  ..., 0.2257, 0.3389, 0.5127],
        [0.0382, 0.0149, 0.0539,  ..., 0.0907, 0.2679, 0.0288],
        ...,
        [0.0000, 0.0053, 0.0000,  ..., 0.2391, 0.1173, 0.1588],
        [0.0000, 0.0053, 0.0000,  ..., 0.1862, 0.2959, 0.0739],
        [0.0000, 0.0053, 0.0000,  ..., 0.1495, 0.0753, 0.1055]])

In [16]:
from torch_geometric.data import Data
feature_data = Data(x=x)
torch.save(feature_data, 'output/'+name+"_feature.pt")

In [17]:
from torch_geometric.data import Data
ppi_data = Data(x=x, edge_index=ppi_edge_index)

In [18]:
import scipy
import scipy.sparse as sp

In [19]:
ppi_adj = scipy.sparse.csr_matrix((np.ones(ppi_data.edge_index.shape[1]), ppi_data.edge_index), shape=(ppi_data.num_nodes, ppi_data.num_nodes))
ppi_adj = sys_normalized_adjacency(ppi_adj)
ppi_adj = sparse_mx_to_torch_sparse_tensor(ppi_adj)

In [20]:
torch.save(ppi_adj, 'output/'+name+"_ppi.pt")

In [21]:
ppi_net = ppi_net[['gene1_id','gene2_id']]
ppi_net.rename({"gene1_id":'gene1',"gene2_id":"gene2"},axis=1,inplace=True)
ppi_net.to_csv('output/'+name+"_ppi.csv",index=False)

## Labels数字化处理

In [22]:
label = pd.read_csv("data/pancan_genelist_for_train.tsv",sep='\t')
label = label[label['Hugosymbol'].isin(omics_gene)]
label.reset_index(drop=True,inplace=True)
label['Hugosymbol'] = label['Hugosymbol'].map(gene_id_mapping)
label

,Hugosymbol,Label
0,17,1
1,18,1
2,19,1
3,20,1
4,30,1
...,...,...
1995,10030,0
1996,10032,0
1997,10056,0
1998,10057,0


In [23]:
label_name = list(label['Hugosymbol'])
label_label = list(label['Label'])

In [24]:
label.to_csv('output/'+name+"_label.csv",index=False)

In [25]:
label

,Hugosymbol,Label
0,17,1
1,18,1
2,19,1
3,20,1
4,30,1
...,...,...
1995,10030,0
1996,10032,0
1997,10056,0
1998,10057,0
